<a href="https://colab.research.google.com/github/vfrantc/dehaze_detect/blob/main/evaluate_detection_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Nov  9 12:22:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


# RUN DETECTOR ON UNPROCESSED IMAGES

In [ ]:
!pip install mmcv-full
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .
!pip install Pillow==7.0.0

     |████████████████████████████████| 389 kB 8.7 MB/s 
     |████████████████████████████████| 185 kB 44.2 MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.16-cp37-cp37m-linux_x86_64.whl size=43656489 sha256=ac153558bf9da8ab5dc9856e228443ad83be58d910c5ee51ce3e1d35febf0991
  Stored in directory: /root/.cache/pip/wheels/ea/0a/1e/95428dfef725d66e6693b61180b928c293489b6d13df3ae0e2
Successfully built mmcv-full
Cloning into 'mmdetection'...
remote: Enumerating objects: 21578, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 21578 (delta 4), reused 4 (delta 1), pack-reused 21559
Receiving objects: 100% (21578/21578), 25.08 MiB | 26.02 MiB/s, done.
Resolving deltas: 100% (15140/15140), done.
/content/mmdetection
Obtaining file:///content/mmdetection
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15354 sha256=7da3b8ce2f8ee16837d7b4f84e6ad01b99cb72ec5016dd20a20ee540877c0ed4
  S

In [ ]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/scnet/scnet_x101_64x4d_fpn_20e_coco/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth \
      -O checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth

--2021-11-09 12:51:02--  https://download.openmmlab.com/mmdetection/v2.0/scnet/scnet_x101_64x4d_fpn_20e_coco/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.59
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611860651 (584M) [application/octet-stream]
Saving to: ‘checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth’

checkpoints/scnet_x 100%[===================>] 583.52M  10.3MB/s    in 65s     

2021-11-09 12:52:07 (8.98 MB/s) - ‘checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth’ saved [611860651/611860651]



In [ ]:
%cd mmdetection

[Errno 2] No such file or directory: 'mmdetection'
/content/mmdetection


In [ ]:
from glob import glob
import os

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.datasets import build_dataloader
from mmdet.apis import single_gpu_test
from mmcv.parallel import MMDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

config = 'configs/scnet/scnet_x101_64x4d_fpn_20e_coco.py'
checkpoint = 'checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth'
model = init_detector(config, checkpoint, device='cuda:0')
model = MMDataParallel(model, device_ids=[0])

Use load_from_local loader


In [ ]:
def create_dataset(folder):
  cfg = Config.fromfile('./configs/scnet/scnet_x101_64x4d_fpn_20e_coco.py')
  cfg.dataset_type = 'CocoDataset'
  PREFIX = folder
  cfg.data.test.img_prefix = folder
  cfg.data.test.ann_file = os.path.join(PREFIX, 'new_test.json')
  cfg.data.test.type = 'CocoDataset'
  dataset = build_dataset(cfg.data.test)
  #dataset.CLASSES = classes
  #dataset.cat_ids = list(range(len(dataset.CLASSES)))
  data_loader = build_dataloader(dataset, samples_per_gpu=1, workers_per_gpu=2, dist=False, shuffle=False)
  return dataset, data_loader

In [ ]:
!cp /content/drive/MyDrive/hazy_detect/out_gca.zip .
!cp /content/drive/MyDrive/hazy_detect/dcp_coco.zip . 
!cp /content/drive/MyDrive/hazy_detect/out_refined.tar.gz .
!cp /content/drive/MyDrive/hazy_detect/dehazenet_coco.zip .
!cp /content/drive/MyDrive/hazy_detect/ide_coco.zip .
!cp /content/drive/MyDrive/hazy_detect/out_epdn.tar.gz .

In [ ]:
!cp /content/drive/MyDrive/new_detect/aod_coco.zip . 
!unzip -q aod_coco.zip

In [ ]:
!unzip -q out_gca.zip
!unzip -q dcp_coco.zip
!tar xzvf out_refined.tar.gz
!unzip -q dehazenet_coco.zip
!unzip -q ide_coco.zip
!tar xzvf out_epdn.tar.gz

out_refined/
out_refined/new_test.json
out_refined/JPEGImages/
out_refined/JPEGImages/BD_Baidu_338_dehz.png
out_refined/JPEGImages/AM_Bing_640_dehz.png
out_refined/JPEGImages/BD_Google_396_dehz.png
out_refined/JPEGImages/BD_Baidu_129_dehz.png
out_refined/JPEGImages/BD_Google_598_dehz.png
out_refined/JPEGImages/BD_Baidu_081_dehz.png
out_refined/JPEGImages/BJ_Baidu_1039_dehz.png
out_refined/JPEGImages/BJ_Baidu_1158_dehz.png
out_refined/JPEGImages/BD_Baidu_133_dehz.png
out_refined/JPEGImages/BD_Bing_303_dehz.png
out_refined/JPEGImages/BD_Bing_664_dehz.png
out_refined/JPEGImages/BD_Baidu_000_dehz.png
out_refined/JPEGImages/BD_Baidu_116_dehz.png
out_refined/JPEGImages/BD_Google_378_dehz.png
out_refined/JPEGImages/BD_Baidu_022_dehz.png
out_refined/JPEGImages/BD_Baidu_102_dehz.png
out_refined/JPEGImages/BD_Baidu_428_dehz.png
out_refined/JPEGImages/BJ_Baidu_095_dehz.png
out_refined/JPEGImages/BD_Google_249_dehz.png
out_refined/JPEGImages/AM_Google_244_dehz.png
out_refined/JPEGImages/BD_Baidu_4

### gca

In [ ]:
!cp /content/drive/MyDrive/new_test.json out_gca

In [ ]:
method = 'out_gca'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
outputs = single_gpu_test(model, data_loader, False, '', 0.3)

loading annotations into memory...
Done (t=0.52s)
creating index...
index created!
[                                                  ] 0/4320, elapsed: 0s, ETA:

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2255s, ETA:     0s

In [ ]:
# how to post-process, and how to change the json

In [ ]:
# post-process the detection to make sure that we use only the correct ones

In [ ]:
dataset.cat_ids.append(91)
metric = dataset.evaluate(outputs, metric='bbox')


Evaluating bbox...
Loading and preparing results...
DONE (t=1.30s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=75.03s).
Accumulating evaluation results...
DONE (t=9.73s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.368
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.576
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.396
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.350
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.489
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.450
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.450
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.450
 Average Recall     (AR) @[ IoU=0.50:0.95

In [ ]:
!tar czf detected_gca.tar.gz out_out_gca/
!cp detected_gca.tar.gz /content/drive/MyDrive/new_detect

### dcp


In [ ]:
method = 'dcp_coco'
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2258s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.67s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=77.50s).
Accumulating evaluation results...
DONE (t=9.68s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.360
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.568
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.387
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.339
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.487
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.438
 Average Recall     (AR) @[ IoU=0.50:0.9

In [ ]:
# !tar czf detected_dcp.tar.gz out_dcp_coco/
# !cp -r detected_dcp.tar.gz /content/drive/MyDrive/new_detect

### Refined

In [ ]:
!cp /content/drive/MyDrive/new_test.json out_refined

In [ ]:
!ls out_refined/JPEGImages

AM_Bing_211_dehz.png	BD_Baidu_18_dehz.png   BD_Bing_606_dehz.png
AM_Bing_217_dehz.png	BD_Baidu_195_dehz.png  BD_Bing_609_dehz.png
AM_Bing_222_dehz.png	BD_Baidu_196_dehz.png  BD_Bing_634_dehz.png
AM_Bing_229_dehz.png	BD_Baidu_199_dehz.png  BD_Bing_643_dehz.png
AM_Bing_232_dehz.png	BD_Baidu_200_dehz.png  BD_Bing_655_dehz.png
AM_Bing_242_dehz.png	BD_Baidu_201_dehz.png  BD_Bing_660_dehz.png
AM_Bing_243_dehz.png	BD_Baidu_209_dehz.png  BD_Bing_664_dehz.png
AM_Bing_274_dehz.png	BD_Baidu_211_dehz.png  BD_Bing_732_dehz.png
AM_Bing_318_dehz.png	BD_Baidu_216_dehz.png  BD_Bing_742_dehz.png
AM_Bing_422_dehz.png	BD_Baidu_222_dehz.png  BD_Bing_74_dehz.png
AM_Bing_464_dehz.png	BD_Baidu_224_dehz.png  BD_Bing_763_dehz.png
AM_Bing_536_dehz.png	BD_Baidu_247_dehz.png  BD_Bing_789_dehz.png
AM_Bing_640_dehz.png	BD_Baidu_249_dehz.png  BD_Bing_802_dehz.png
AM_Bing_727_dehz.png	BD_Baidu_250_dehz.png  BD_Google_00_dehz.png
AM_Google_019_dehz.png	BD_Baidu_254_dehz.png  BD_Google_054_dehz.png
AM_Google_035_dehz.pn

In [ ]:
method = 'out_refined'
!rm -rf out_out_refined
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
[                                                  ] 0/4320, elapsed: 0s, ETA:

FileNotFoundError: ignored

In [ ]:
# !tar czf detected_refined.tar.gz out_out_refined/
# !cp -r detected_refined.tar.gz /content/drive/MyDrive/new_detect

# Dehazenet

In [17]:
method = 'dehazenet_coco'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

loading annotations into memory...
Done (t=0.24s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2236s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.89s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=79.36s).
Accumulating evaluation results...
DONE (t=9.47s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.356
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.570
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.381
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.338
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.482
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.436
 Average Recall     (AR) @[ IoU=0.50:0.9

In [ ]:
# !tar czf detected_dehazenet.tar out_dehazenet/
# !cp -r detected_dehazenet.zip /content/drive/MyDrive/new_detect

# IDE

In [18]:
method = 'dide_coco'
!cp /content/drive/MyDrive/new_test.json dide_coco
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2263s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.95s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=77.21s).
Accumulating evaluation results...
DONE (t=9.73s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.359
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.569
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.388
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.141
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.487
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.442
 Average Recall     (AR) @[ IoU=0.50:0.9

In [ ]:
# !tar czf detected_ide.tar.gz out_ide_coco/
# !cp -r detected_ide.tar.gz /content/drive/MyDrive/new_detect

# RTTS

In [22]:
!ls /content/

drive  mmdetection  sample_data


In [23]:
!rm -rf rtts_coco
!cp /content/drive/MyDrive/haze/data/RTTS.zip .
!unzip -q RTTS.zip
!mv RTTS rtts_coco 
!cp /content/drive/MyDrive/new_test.json rtts_coco

In [24]:
method = 'rtts_coco'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

mkdir: cannot create directory ‘out_rtts_coco’: File exists
loading annotations into memory...
Done (t=0.20s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2235s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.87s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=77.61s).
Accumulating evaluation results...
DONE (t=9.41s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.359
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.571
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.383
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.486
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | ma

In [ ]:
# !tar czf detected_rtts_coco.tar.gz out_rtts_coco/
# !cp -r detected_rtts_coco.tar.gz /content/drive/MyDrive/new_detect

# AOD

In [25]:
method = 'aod_coco'
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

loading annotations into memory...
Done (t=0.88s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2279s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.95s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=83.48s).
Accumulating evaluation results...
DONE (t=9.90s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.338
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.547
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.359
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.131
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.320
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.463
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.420
 Average Recall     (AR) @[ IoU=0.50:0.9

# EPDN

In [28]:
!ls out_epdn/JPEGImages

AM_Bing_211_final.png	     hv36_639_final.png
AM_Bing_217_final.png	     hv36_652_final.png
AM_Bing_222_final.png	     hv36_670_final.png
AM_Bing_229_final.png	     hv36_675_final.png
AM_Bing_232_final.png	     hv36_68_final.png
AM_Bing_242_final.png	     hv36_87_final.png
AM_Bing_243_final.png	     hv36_92_final.png
AM_Bing_274_final.png	     hv37_1022_final.png
AM_Bing_318_final.png	     hv37_1076_final.png
AM_Bing_422_final.png	     hv37_1088_final.png
AM_Bing_464_final.png	     hv37_1095_final.png
AM_Bing_536_final.png	     hv37_1107_final.png
AM_Bing_640_final.png	     hv37_1136_final.png
AM_Bing_727_final.png	     hv37_115_final.png
AM_Google_019_final.png      hv37_1165_final.png
AM_Google_035_final.png      hv37_1200_final.png
AM_Google_067_final.png      hv37_1222_final.png
AM_Google_10_final.png	     hv37_1246_final.png
AM_Google_228_final.png      hv37_1260_final.png
AM_Google_244_final.png      hv37_1275_final.png
AM_Google_246_final.png      hv37_1294_final.png
AM_Google_2

In [26]:
method = 'out_epdn'
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
[                                                  ] 0/4320, elapsed: 0s, ETA:

FileNotFoundError: ignored